![JohnSnowLabs](https://sparknlp.org/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp/blob/master/examples/python/transformers/onnx/HuggingFace_ONNX_in_Spark_NLP_InstructorEmbeddings.ipynb)

# Import ONNX InstructorEmbeddings  models from HuggingFace 🤗 into Spark NLP 🚀

Let's keep in mind a few things before we start 😊

- ONNX support was introduced in `Spark NLP 5.1.0`, enabling high performance inference for models. Please make sure you have upgraded to the latest Spark NLP release.


## Export and Save HuggingFace model

- Let's install `transformers` package with the `onnx` extension and it's dependencies. You don't need `onnx` to be installed for Spark NLP, however, we need it to load and save models from HuggingFace.
- We lock `transformers` on version `4.31.0`. This doesn't mean it won't work with the future releases, but we wanted you to know which versions have been tested successfully.

In [1]:
!pip install sentence-transformers
!pip install -q --upgrade "transformers[onnx]===4.39.3" optimum

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 41.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 421.5/421.5 kB 22.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 60.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.7/212.7 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 58.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 21.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.5/84.5 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 35.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 455.8/455.8 kB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.

- HuggingFace has an extension called Optimum which offers specialized model inference, including ONNX. We can use this to import and export ONNX models with `from_pretrained` and `save_pretrained`.
- We'll use the [hkunlp/instructor-base](https://huggingface.co/hkunlp/instructor-base) model from HuggingFace as an example and export it with the `optimum-cli`.

In [2]:
MODEL_NAME = "hkunlp/instructor-base"
EXPORT_PATH = f"export_onnx/{MODEL_NAME}"

In [3]:
! optimum-cli export onnx --model {MODEL_NAME} {EXPORT_PATH} --task feature-extraction

2024-09-07 20:07:27.251422: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-07 20:07:27.293799: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-07 20:07:27.305820: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-07 20:07:29.013748: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Framework not specified. Using pt to export the model.
modules.json: 100% 461/461 [00:00<00:00, 1.77MB/s]
config_sentence_transformers.json: 100% 122/122 [00:00<00:00, 619kB/s]
README.md: 100% 66.2k/66.2k [00:00<00:00, 3.25MB/s]
sentence_ber

In [4]:
! mkdir -p {EXPORT_PATH}/assets
! mv -t {EXPORT_PATH}/assets  {EXPORT_PATH}/*.model

Let's have a look inside these two directories and see what we are dealing with:

In [5]:
!ls -l {EXPORT_PATH}

total 433156
drwxr-xr-x 2 root root      4096 Sep  7 20:08 assets
-rw-r--r-- 1 root root      1545 Sep  7 20:07 config.json
-rw-r--r-- 1 root root 441088928 Sep  7 20:08 model.onnx
-rw-r--r-- 1 root root      2543 Sep  7 20:07 special_tokens_map.json
-rw-r--r-- 1 root root     20937 Sep  7 20:07 tokenizer_config.json
-rw-r--r-- 1 root root   2422456 Sep  7 20:07 tokenizer.json


In [6]:
!ls -l {EXPORT_PATH}/assets

total 776
-rw-r--r-- 1 root root 791656 Sep  7 20:07 spiece.model


## Import and Save InstructorEmbeddings  in Spark NLP

- Let's install and setup Spark NLP in Google Colab
- This part is pretty easy via our simple script
- Additionally, we need to upgrade Spark to version 3.4.1.

In [7]:
! wget -q http://setup.johnsnowlabs.com/colab.sh -O - | bash
! pip install -U pyspark==3.4.1

Installing PySpark 3.2.3 and Spark NLP 5.4.2
setup Colab for PySpark 3.2.3 and Spark NLP 5.4.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.5/281.5 MB 5.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.6/55.6 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 579.5/579.5 kB 32.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached py4j-0.10.9.7-py2.py3-none-any.whl.metadata (1.5 kB)
Using cached py4j-0.10.9.7-py2.py3-none-any.whl (200 kB)
  Created wheel for pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285391 sha256=eb9327e6858e30ee1b29ea2a8e61c63031806f96549ddbfbe0942556c2b89978
  Stored in directory: /root/.cache/pip/wheels/0d/77/a3/ff2f74cc9ab41f8f594dabf0579c2a7c6de920d584206e0834
Successfully built pyspar

Let's start Spark with Spark NLP included via our simple `start()` function

In [8]:
import sparknlp
# let's start Spark with Spark NLP
spark = sparknlp.start()
print("Apache Spark version: {}".format(spark.version))

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 629.6/629.6 kB 12.9 MB/s eta 0:00:00
  Attempting uninstall: spark-nlp
    Found existing installation: spark-nlp 5.4.2
    Uninstalling spark-nlp-5.4.2:
      Successfully uninstalled spark-nlp-5.4.2


- Let's use `loadSavedModel` functon in `InstructorEmbeddings ` which allows us to load the ONNX model
- Most params will be set automatically. They can also be set later after loading the model in `InstructorEmbeddings ` during runtime, so don't worry about setting them now
- `loadSavedModel` accepts two params, first is the path to the exported model. The second is the SparkSession that is `spark` variable we previously started via `sparknlp.start()`
- NOTE: `loadSavedModel` accepts local paths in addition to distributed file systems such as `HDFS`, `S3`, `DBFS`, etc. This feature was introduced in Spark NLP 4.2.2 release. Keep in mind the best and recommended way to move/share/reuse Spark NLP models is to use `write.save` so you can use `.load()` from any file systems natively.st and recommended way to move/share/reuse Spark NLP models is to use `write.save` so you can use `.load()` from any file systems natively.

In [14]:
from sparknlp.annotator import *
from sparknlp.base import *

embedding = InstructorEmbeddings.loadSavedModel(
     EXPORT_PATH,
     spark
 )\
  .setInputCols(["document"])\
  .setOutputCol("instructor")

- Let's save it on disk so it is easier to be moved around and also be used later via `.load` function

In [15]:
embedding.write().overwrite().save("./{}_spark_nlp".format(EXPORT_PATH))

Awesome  😎 !

This is your ONNX InstructorEmbeddings  model from HuggingFace 🤗  loaded and saved by Spark NLP 🚀

In [16]:
! ls -l {EXPORT_PATH}_spark_nlp

total 431604
-rw-r--r-- 1 root root 441156367 Sep  7 20:11 instructor_onnx
-rw-r--r-- 1 root root    791656 Sep  7 20:11 instructor_spp
drwxr-xr-x 2 root root      4096 Sep  7 20:11 metadata


Now let's see how we can use it on other machines, clusters, or any place you wish to use your new and shiny InstructorEmbeddings  model 😊

In [18]:
import sparknlp

from sparknlp.base import *
from sparknlp.annotator import *

document_assembler = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

instructor_loaded = InstructorEmbeddings.load(f"{EXPORT_PATH}_spark_nlp")\
    .setInputCols(["document"])\
    .setOutputCol("instructor")\
    .setInstruction("Encode This:")

pipeline = Pipeline(
    stages = [
        document_assembler,
        instructor_loaded
  ])

data = spark.createDataFrame([['William Henry Gates III (born October 28, 1955) is an American business magnate, software developer, investor,and philanthropist.']]).toDF("text")
model = pipeline.fit(data)
result = model.transform(data)
result.show()

+--------------------+--------------------+--------------------+
|                text|            document|          instructor|
+--------------------+--------------------+--------------------+
|William Henry Gat...|[{document, 0, 12...|[{sentence_embedd...|
+--------------------+--------------------+--------------------+



That's it! You can now go wild and use hundreds of InstructorEmbeddings  models from HuggingFace 🤗 in Spark NLP 🚀
